In [11]:
import pandas as pd
from scipy.sparse import csr_matrix
from pandas.api.types import CategoricalDtype

In [24]:
tr_long = pd.read_csv("./data/preproc/stg2up_2d_full_trainX.csv")
tr_y = pd.read_csv("./data/preproc/stg2up_2d_full_trainy.csv")
ts_long = pd.read_csv("./data/preproc/stg2up_2d_full_testX.csv")
ts_y = pd.read_csv("./data/preproc/stg2up_2d_full_testy.csv")

In [13]:
print(tr_long.head(5))

    ROW_ID  dsa         key  value
0  10000_1 -1.0         AGE   68.0
1  10000_1 -1.0       SEX_F    1.0
2  10000_1 -1.0     RACE_05    1.0
3  10000_1 -1.0  HISPANIC_N    1.0
4  10000_1  0.0        day0    1.0


convert training data from long to wide using pd.pivot function
ref: https://stackoverflow.com/questions/31661604/efficiently-create-sparse-pivot-tables-in-pandas

In [ ]:
#dat_wide = dat_long.pivot('ROW_ID','key') #duplicates exist

#take the latest value for each (ROW_ID,key)
tr_long = tr_long.loc[tr_long.groupby(['ROW_ID','key'])['dsa'].idxmin()]
del tr_long['dsa']

#instead of using pivot, we will convert dat_long into sparse Matrix
ROW_ID_c = CategoricalDtype(sorted(tr_long.ROW_ID.unique()), ordered=True)
key_c = CategoricalDtype(sorted(tr_long.key.unique()), ordered=True)

row = tr_long.ROW_ID.astype(ROW_ID_c).cat.codes
col = tr_long.key.astype(key_c).cat.codes
sparse_mt_tr = csr_matrix((tr_long["value"], (row, col)), \
                           shape=(ROW_ID_c.categories.size, key_c.categories.size))

#print out sparse matrix dimension
sparse_mt_tr

#sort tr_y by ROW_ID in sparse_mt_tr
#ref: https://stackoverflow.com/questions/23482668/sorting-by-a-custom-list-in-pandas
tr_y = tr_y[ROW_ID_c,'y']

Now, we can convert testing data from long to wide using pd.pivot function and reuse key_c from training,
so that columns of training and testing data are in line

In [ ]:
#take the latest value for each (ROW_ID,key)
ts_long = ts_long.loc[dat_long.groupby(['ROW_ID','key'])['dsa'].idxmin()]

#instead of using pivot, we will convert ts_long into sparse Matrix
ROW_ID_c = CategoricalDtype(sorted(ts_long.ROW_ID.unique()), ordered=True)

row = ts_long.ROW_ID.astype(ROW_ID_c).cat.codes
col = ts_long.key.astype(key_c).cat.codes
sparse_mt_ts = csr_matrix((ts_long["value"], (row, col)), \
                           shape=(ROW_ID_c.categories.size, key_c.categories.size))

#print out sparse matrix dimension
sparse_mt_ts

#sort ts_y by ROW_ID in sparse_mt_ts
ts_y = ts_y[ROW_ID_c,'y']

As suggested by https://stackoverflow.com/questions/40817459/xgboost-and-sparse-matrix, sparse_mt_tr and tr_y can be directly used in xgboost model. 